# 1. ВЫБОР ДАТАСЕТА

Для реализации домашней работы по анализу данных по курсу "Упорядоченные множества в анализе данных" был выбран датасет с чемпионата kaggle https://www.kaggle.com/c/GiveMeSomeCredit/data, в котором предлагается решить задачу кредитного скоринга.

Количество объектов: 150000

Количество признаков: 8

Краткое описание признаков:

SeriousDlqin2yrs - Person experienced 90 days past due delinquency or worse - Y/N
RevolvingUtilizationOfUnsecuredLines - Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits	(percentage)
age	Age of borrower in years (integer)

NumberOfTime30-59DaysPastDueNotWorse - Number of times borrower has been 30-59 days past due but no worse in the last 2 years	(integer)

DebtRatio - Monthly debt payments, alimony,living costs divided by monthy gross income	percentage
MonthlyIncome - Monthly income (real)

NumberOfOpenCreditLinesAndLoans	- Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards)	(integer)

NumberOfTimes90DaysLate - Number of times borrower has been 90 days or more past due	(integer)

NumberRealEstateLoansOrLines - Number of mortgage and real estate loans including home equity lines of credit	(integer)

NumberOfTime60-89DaysPastDueNotWorse - Number of times borrower has been 60-89 days past due but no worse in the last 2 years	(integer)

NumberOfDependents - Number of dependents in family excluding themselves (spouse, children etc.)	(integer)




# 2. ПЕРЕСМОТР ПРОСТРАНСТВА ПРИЗНАКОВ В ВЫБРАННОМ ДАТАСЕТЕ

Импортируем датасет

In [5]:
import pandas as pd
data = pd.read_csv('cs-training.csv')

Исключение из анализа объектов с пустыми значениями (NAN)

In [6]:
data = data.dropna()

Перед преобразованием были построены гистограмы, и признаки имеющие показательное распределение и небольшое число значений были разделены по первым пяти значения [x1,x1], [x2,x2],... Хвост показательного распределение был усечен. Признаки имеющие нормальное распределение или большое число значений были преобразованы с помощью квантилей.

In [7]:
#Категоризация признаков с нормальным распредлением
data['RevolvingUtilizationOfUnsecuredLines_c'] = pd.qcut(data['RevolvingUtilizationOfUnsecuredLines'], 8)
data['age_c'] = pd.qcut(data['age'], 8)
data['DebtRatio_c'] = pd.qcut(data['DebtRatio'], 8)
data['MonthlyIncome_c'] = pd.qcut(data['MonthlyIncome'], 8)
data['NumberOfOpenCreditLinesAndLoans_с'] = pd.qcut(data['NumberOfOpenCreditLinesAndLoans'], 8)
#Категоризация признаков со степенным распредлеением
data.ix[data['NumberOfTime30-59DaysPastDueNotWorse']>5, 'NumberOfTime30-59DaysPastDueNotWorse'] = 5
data.ix[data['NumberOfTimes90DaysLate']>5, 'NumberOfTimes90DaysLate'] = 5
data.ix[data['NumberRealEstateLoansOrLines']>5, 'NumberRealEstateLoansOrLines'] = 5
data.ix[data['NumberOfTime60-89DaysPastDueNotWorse']>5, 'NumberOfTime60-89DaysPastDueNotWorse'] = 5
data.ix[data['NumberOfDependents']>5, 'NumberOfDependents'] = 5

Удаление столбцов с количественными признаками + удаление столбца ID

In [8]:
data = data.drop(['Unnamed: 0', 'RevolvingUtilizationOfUnsecuredLines','age', 'DebtRatio','MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans'], 1)

Преобразование категориальных признаков в бинарные

In [10]:
data = pd.get_dummies(data, columns = ['NumberOfOpenCreditLinesAndLoans_с','RevolvingUtilizationOfUnsecuredLines_c','age_c', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio_c','MonthlyIncome_c','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents'])

Сохранение нового датасета

In [11]:
data.to_csv('credit_new.csv',index=False)

# 3. КРОСС-ВАЛИДАЦИЯ

Проведение 10 блочной кросс-валидация

In [12]:
newdata = pd.read_csv('2008_200_300_new.csv')

from sklearn.cross_validation import KFold

kf = KFold(len(newdata), n_folds=10, shuffle=True, random_state=None)
for k, (train, test) in enumerate(kf):
    newdata.iloc[train].to_csv('train'+str(k+1)+'.csv',index=False)
    newdata.iloc[test].to_csv('test'+str(k+1)+'.csv',index=False)

# 4. РЕАЛИЗАЦИЯ ВСПОМОГАТЕЛЬНЫХ ФУНКЦИЙ

Функция загрузки train и test и их дальнейшего разделение на целевую функцию и множество признаков

In [ ]:
def parse_file(name):
    df = pd.read_csv(name, sep=',')
    y = np.array(df['SeriousDlqin2yrs'])
    del df['SeriousDlqin2yrs']
    return np.array(df).astype(int), y

# 5. РЕАЛИЗАЦИЯ АЛГОРИТМОВ

Первоначально за работу были взят алгоритм 2 из работы Сукмановой Елены (https://github.com/fdrstrok/lazyfca15/blob/master/sukmanova_lena/lazyfca15_project_Sukmanova.ipynb), показавший наилучшее значение различных метрик на датасете с медецинскими данными. Однако при его тестирование результаты получены не были, так как он работает крайне медленно. 

Поэтому необходимо отказаться от стандартной схемы ленивой классификации (поиска вложений пересечения классифицируемого объекта и плюс контекста в минус контексте и наоборот). В качестве альтернативы рассмотрим вариант алгоритма голосования при сопадение плюс контекста и классифицируемого объекта (тоже самое для минуса). 

Кроме этого, рассмотрим вариант с проведением нормализации числа голосов за плюс и минус с помощью кол-ва данных объектов в обучающей выборке. 

Также рассмотрим вариант с эмпирически подобранным коэфициентом, корректирующем голосование за плюс и минус контекст (коэфициент равен 0,91). 

Проверку работы алгоритма проведем на полноценной обучающей выборке для 200 тестовых объектов из примерно 6000 в каждом из 10 файлов с тестовыми выборками, полученных в результате кросс-валидации.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def pred_data(i, part):
    X_train, y_train = parse_file('train' + str(i) + '.csv')
    X_test, y_test = parse_file('test' + str(i) + '.csv')
    X_train_pos = X_train[y_train == 1]
    X_train_neg = X_train[y_train == 0]
    
    y_pred = [] 
    
    k=1   
    for test_obj in X_test:
        #Печать % классифицированных объектов
        print(i, k*100/len(X_test))
        k+=1
        
        pos = 0
        neg = 0
        
        for pos_obj in X_train_pos:
            #part - коэфициент определяющий больше какой части от длины позитивных объектов должно быть число совпадений
            if np.sum(test_obj == pos_obj)> int(len(pos_obj)*part):
                pos += 1
                
        for neg_obj in X_train_neg:
            #part - коэфициент определяющий больше какой части от длины негативных объектов должно быть число совпадений
            if np.sum(test_obj == neg_obj)  > int(len(neg_obj)*part):
                neg += 1
        #Нормирование позитивных и негативных голосов на их кол-во в обучающей выборке           
        pos = pos / float(len(X_train_pos))     
        neg = neg / float(len(X_train_neg))

        if (pos > neg):
            y_pred.append(1)
        else:
            y_pred.append(0)
            
    y_pred = np.array(y_pred)

    #метрики качества
    TP = np.sum(y_test * y_pred)
    TN = np.sum(y_test + y_pred == 0)
    FP = np.sum((y_test  == 0) * (y_pred == 1))
    FN = np.sum((y_test  == 1) * (y_pred == 0))
    TPR = float(TP) / np.sum(y_test == 1)
    TNR = float(TN) / np.sum(y_test == 0)
    FPR = float(FP) / (TP + FN)
    NPV = float(TN) / (TN + FN)
    FDR = float(FP) / (TP + FP)
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    
    print("Dataset {}".format(i))
    #print("True Positive: {}\nTrue Negative: {}\nFalse Positive: {}\nFalse Negative: {}\nTrue Positive Rate: {}\nTrue Negative Rate: {}\nNegative Predictive Value: {}\nFalse Positive Rate: {}\nFalse Discovery Rate: {}\nAccuracy: {}\nPrecision: {}\nRecall: {}".format(TP, TN, FP, FN, TPR, TNR, FPR, NPV, FDR, acc, prec, rec)
    print("Accuracy: {}\nPrecision: {}\nRecall: {}\nTime: {}".format(acc, prec, rec, time))
    print("===========")

Запускаем алгоритм

In [ ]:
start = timeit.default_timer()
for i in range(0, 1):
    acc, prec, rec = pred_data(i+1, 0.91)
    acc1.append(acc)
    prec1.append(prec)
    rec1.append(rec)
stop = timeit.default_timer()
time = stop - start
print("Accuracy: {}\nPrecision: {}\nRecall: {}\nTime: {}".format(sum(acc1)/len(acc1), sum(prec1)/len(prec1),sum(rec1)/len(rec1), time))

Результаты без нормирование:

In [ ]:
Accurcy: 61.2349432602
Precision: 45.7845124232
Recall: 27.3244591821
Time: 125.957654657

Результаты с нормированием:

In [ ]:
Accurcy: 67.345245911
Precision: 48.561112245
Recall: 40.122197542
Time: 125.768896567

In [ ]:
Так как результат с нормирование лучше, то используя его проверим эффективность использования коэфициента:

In [ ]:
Accurcy: 79.235432434
Precision: 81.345124737
Recall: 76.365651864
Time: 125.124435457

In [ ]:
С нормированием и без коэфициента:

In [ ]:
Accurcy: 74.673242499
Precision: 76.562359145
Recall: 69.791203422
Time: 125.76889656

# Работа с узорными структурами

Рассмотренный выше алгоритм был преобразован чтобы работать с узорными структурами. Однако для корректной работы в данном пространстве признаков необходима полноценная проверка пересечения классифицированного объекта с плюс контекстом в минус контексте. В результате этого время работы алгоритма значительно возрастает и проверка его работы на данных скоринга с 150 тыс. объектов будет очень медленно. Поэтому алгоритм был испытан на данных из работы Сукмановой Елены (медецинских). 

In [ ]:
def pred_data_uzor(i, part):
    X_train, y_train = parse_file('train' + str(i) + '.csv')
    X_test, y_test = parse_file('test' + str(i) + '.csv')
    X_train_pos = X_train[y_train == 1]
    X_train_neg = X_train[y_train == 0]
    
    y_pred = [] 
    k=1   
    for test_obj in X_test:
        print(k*100/len(X_test))
        k+=1
        pos = 0
        neg = 0
    
        for pos_obj in X_train_pos:
            counter_pos=0
            pos += 1
            for neg_obj in X_train_neg:
                if (neg_obj>=np.minimum(pos_obj,test_obj)).all() and (neg_obj<=np.maximum(pos_obj,test_obj)).all():
                    counter_pos += 1
                    if counter_pos>1:
                        pos-=1
                        break       
                
        for neg_obj in X_train_neg:
            counter_neg=0
            neg += 1
            for pos_obj in X_train_pos:
                if (pos_obj>=np.minimum(test_obj,neg_obj)).all() and (pos_obj<=np.maximum(test_obj,neg_obj)).all():
                    counter_neg += 1
                    if counter_neg>1:
                        neg-=1
                        break
                    
        pos = pos / float(len(X_train_pos))
        neg = neg / float(len(X_train_neg))
        if (pos > neg):
            y_pred.append(1)
        else:
            y_pred.append(0)
            
    y_pred = np.array(y_pred)
    #print y_pred
    
    #метрики качества
    TP = np.sum(y_test * y_pred)
    TN = np.sum(y_test + y_pred == 0)
    FP = np.sum((y_test  == 0) * (y_pred == 1))
    FN = np.sum((y_test  == 1) * (y_pred == 0))
    TPR = float(TP) / np.sum(y_test == 1)
    TNR = float(TN) / np.sum(y_test == 0)
    FPR = float(FP) / (TP + FN)
    NPV = float(TN) / (TN + FN)
    FDR = float(FP) / (TP + FP)
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    
    print("Dataset {}".format(i))
    #print("True Positive: {}\nTrue Negative: {}\nFalse Positive: {}\nFalse Negative: {}\nTrue Positive Rate: {}\nTrue Negative Rate: {}\nNegative Predictive Value: {}\nFalse Positive Rate: {}\nFalse Discovery Rate: {}\nAccuracy: {}\nPrecision: {}\nRecall: {}".format(TP, TN, FP, FN, TPR, TNR, FPR, NPV, FDR, acc, prec, rec)
    print("Accuracy: {}\nPrecision: {}\nRecall: {}\nTime: {}".format(sum(acc1)/len(acc1), sum(prec1)/len(prec1),sum(rec1)/len(rec1), time))
    print("===========")

Запускаем алгоритм:

In [ ]:
start = timeit.default_timer()
for i in range(0, 1):
    acc, prec, rec = pred_data(i+1, 0.91)
    acc1.append(acc)
    prec1.append(prec)
    rec1.append(rec)
stop = timeit.default_timer()
time = stop - start
print("Accuracy: {}\nPrecision: {}\nRecall: {}\nTime: {}".format(sum(acc1)/len(acc1), sum(prec1)/len(prec1),sum(rec1)/len(rec1), time))

Результат работы:

In [ ]:
Accuracy: 0.72
Precision: 0.8333333333333334
Recall: 0.7142857142857143
Time: 4563.84799351335005

Заметим, что узорные структуры в сочетании с алгоритмом голосования сработали хуже чем Алгоритм 2 по метрикам точность и полнота из работы Сукмановой Елены:

In [ ]:
Accurcy: 77.01981981981983
Precision: 80.6285098120081
Recall: 86.90841576918024
Time: 8877.83866877500

Однако в данной работе не производилось дополнительной "подгонки" с помощью различных коэфициентов. Лучше оказалось только значение полноты (на 3%) и значение времени 4500 против 9000, что обусловлено использованием разных конструкций языка Python.

Также не был проверен вариант без голосования.

# Параллельное вычисление

Для данной задачи была использована библиотека multiprocessing. Организуем параллельное вычисление, для ускорение работы ленивого классификатора. В данном примере пареллелизация реализована для выборок получившихся в результате кроcс-валидации. Также параллельно можно вычислять вложения в плюс минус контекст, так как они происходят независимо.

In [ ]:
if __name__ == '__main__':
    start = timeit.default_timer()
    #вычисляем количество ядер
    worker_count = multiprocessing.cpu_count()
    jobs = []
    #инициализируем классификатор на нескольких тестовых и обучающих выборках получившихся в результате кросвалидации
    #(одновременно будет классифицироваться число выборк равное числу ядер) 
    for i in range(worker_count):
        p = multiprocessing.Process(target=pred_data, args=(i+1, 0.91))
        jobs.append(p)
        p.start()
    # ожидаем завершения работы всех воркеров
    for w in jobs:
        w.join()
    stop = timeit.default_timer()
    time = stop - start
    print("Time: {}".format(time))

Результаты работы по времени для 200 объектов 4 тестовых выборок, полученных в результате кросс-валидации:

In [ ]:
Time: 431.3454645345456

Заметим, что это почти в 2,5 быстрее чем обычный запуск алгоритма для 4 подвыборок:

In [ ]:
Time: 1035.2291148834455

# Сравнение с байсовским классификатором

In [3]:
def BernoulliNaiveBayes():
    import timeit
    start = timeit.default_timer()
    
    q = open('2008_200_300_new.csv', 'r')
    dataset = [a.strip().split(',') for a in q]
    dataset = dataset[1:]
    q.close()
    
    import numpy as np
    A = np.array(dataset)
    X = A[:,1:].astype(float)
    Y = A[:,0]
    
    from sklearn.naive_bayes import BernoulliNB
    from sklearn import cross_validation
    from sklearn.preprocessing import LabelBinarizer
    model = BernoulliNB()
    model.fit(X, Y)
    
    acc = np.mean(cross_validation.cross_val_score(model, X, Y, cv=10))
    
    lb = LabelBinarizer()
    Y = np.array([number[0] for number in lb.fit_transform(Y)])
    
    prec = np.mean(cross_validation.cross_val_score(model, X, Y, cv=10, scoring = 'precision'))
    rec = np.mean(cross_validation.cross_val_score(model, X, Y, cv=10, scoring = 'recall'))
    
    stop = timeit.default_timer()
    time = stop - start

    return acc, prec, rec, time

Запускаем алгоритм:

In [ ]:
(aсс,prec,rec,time) = BernoulliNaiveBayes()
print('Accurcy: '+str(acc*100))
print('Precision: '+str(prec*100))
print('Recall: '+str(rec*100))
print('Time: '+str(time))

Результат для 200 объектов тестовой выборки:

In [ ]:
Accurcy: 80.3397260274
Precision: 89.6280133232
Recall: 79.8489795918
Time: 0.2465597490011

# ИТОГИ

1). В результате данной работы, были изучены различные методы ленивой классификации. Для решения задачи скоринга с Kaggle и аналогичных (с большим кол-вом объектов) рекомендуется использовать алгоритм "частичной"ленивой классификации с голосованием, так как он работает намного быстрее полной ленивой классификации. 

2). Кроме того, следует отметить значимость правильной категоризации и дальнейшей бинаризации данных. Было определено, необходимо смотреть на гистограмму каждого признака и подбирать соотвествующую бинаризуцию. В данной работе признаки с нормальным распределеним и/или большим числом значений былы бинаризованы с помощью квантилей, а признаки со степенным распределением и/или малым числом значений разбиты на интервалы [x,x], с усечением "хвоста" степенного распределения.

3). Алгоритмы с голосованием работают эффективнее (показано в работе Сукмановой Елены), а введение дополнительных коэффициентов способно еще больше повысить их точность (показано в данной работе).

4). Узорные структуры в виде числовых интервалов показали себя несколько хуже по некоторым метрикам чем алгоритм с бинаризацией на данных с небольшим числом объектов. Для данных с большим числом они также работают крайне медленно.

5). Мультипроцессинг способен значительно повысить скорость работы классификатора на основе линивых вычислений.

6). "Стандартные" алгоритмы работают быстрее ленивой классификации, что в данной работе было продемонстрировано с помощью сравнения с байсовским классификатором. Кроме того стандартные алгоритмы показывают схожие результаты. Однако алгоритмы ленивой классификации имеют значительные возможности "настройки", благодаря чему теоретически могут превзойти по качеству стандартные, кроме того они более лего интерпитируемы.